In [103]:
print("I'm actually working...") ## Kernel refuses to start sometimes.
import numpy as np
from myuproot import open_up
import uproot3_methods
from tqdm import tqdm

I'm actually working...


In [61]:
f = 'signal/NanoAOD/NMSSM_XYH_YToHH_6b_MX_700_MY_400_reco_preselections.root'
t = open_up(f)
nevents = t._length()
nevents

----------------------------------------------------------------------------------------------------
                                            TABLE COLUMNS                                           
----------------------------------------------------------------------------------------------------
HX_b1_recojet_m                   HX_b1_recojet_pt                  HX_b1_recojet_eta                 
HX_b1_recojet_phi                 HX_b1_recojet_ptRegressed         HX_b2_recojet_m                   
HX_b2_recojet_pt                  HX_b2_recojet_eta                 HX_b2_recojet_phi                 
HX_b2_recojet_ptRegressed         HY1_b1_recojet_m                  HY1_b1_recojet_pt                 
HY1_b1_recojet_eta                HY1_b1_recojet_phi                HY1_b1_recojet_ptRegressed        
HY1_b2_recojet_m                  HY1_b2_recojet_pt                 HY1_b2_recojet_eta                
HY1_b2_recojet_phi                HY1_b2_recojet_ptRegressed        HY2_b1_reco

199792

In [62]:
HX_b1  = {'pt': t[f'HX_b1_recojet_ptRegressed' ],
          'eta':t[f'HX_b1_recojet_eta'],
          'phi':t[f'HX_b1_recojet_phi'],
          'm':  t[f'HX_b1_recojet_m'  ]}#,
#          'btag':  t[f'HX_b1_recojet_deepJet']}
HX_b2  = {'pt': t[f'HX_b2_recojet_ptRegressed' ],
          'eta':t[f'HX_b2_recojet_eta'],
          'phi':t[f'HX_b2_recojet_phi'],
          'm':  t[f'HX_b2_recojet_m'  ]}#,
#          'btag':  t[f'HX_b2_recojet_deepJet']}
HY1_b1 = {'pt': t[f'HY1_b1_recojet_ptRegressed'],
          'eta':t[f'HY1_b1_recojet_eta'],
          'phi':t[f'HY1_b1_recojet_phi'],
          'm':  t[f'HY1_b1_recojet_m' ]}#,
#          'btag':  t[f'HY1_b1_recojet_deepJet']}
HY1_b2 = {'pt': t[f'HY1_b2_recojet_ptRegressed'],
          'eta':t[f'HY1_b2_recojet_eta'],
          'phi':t[f'HY1_b2_recojet_phi'],
          'm':  t[f'HY1_b2_recojet_m' ]}#,
#          'btag':  t[f'HY1_b2_recojet_deepJet']}
HY2_b1 = {'pt': t[f'HY2_b1_recojet_ptRegressed'],
          'eta':t[f'HY2_b1_recojet_eta'],
          'phi':t[f'HY2_b1_recojet_phi'],
          'm':  t[f'HY2_b1_recojet_m' ]}#,
#          'btag':  t[f'HY2_b1_recojet_deepJet']}
HY2_b2 = {'pt': t[f'HY2_b2_recojet_ptRegressed'],
          'eta':t[f'HY2_b2_recojet_eta'],
          'phi':t[f'HY2_b2_recojet_phi'],
          'm':  t[f'HY2_b2_recojet_m' ]}#,
#          'btag':  t[f'HY2_b2_recojet_deepJet']}

part_dict = {0:HX_b1, 1:HX_b2, 2:HY1_b1, 3:HY1_b2, 4:HY2_b1, 5:HY2_b2}
part_name = {0:'HX_b1', 1:'HX_b2', 2:'HY1_b1', 3:'HY1_b2', 4:'HY2_b1', 5:'HY2_b2'}

In [63]:
from kinematics import calcDeltaR

In [64]:
# Save inputs by pair and invariant mass by pair
X_temp = []
m_temp = []

counter = 0
flag = True
for i in range(5):
    for j in range(i+1,6):
        pair_name = part_name[i] + " & " + part_name[j]
        # print(f"Generating features for pair: {pair_name}")
        dR = calcDeltaR(part_dict[i]['eta'], part_dict[j]['eta'], part_dict[i]['phi'], part_dict[j]['phi'])
        inputs = np.column_stack((part_dict[i]['pt'], part_dict[i]['eta'], part_dict[i]['phi'], 
                                  part_dict[j]['pt'], part_dict[j]['eta'], part_dict[j]['phi'], dR))

        b1 = uproot3_methods.TLorentzVectorArray.from_ptetaphim(part_dict[i]['pt'], part_dict[i]['eta'], 
                                                                part_dict[i]['phi'], np.repeat(4e-9, nevents))
        b2 = uproot3_methods.TLorentzVectorArray.from_ptetaphim(part_dict[j]['pt'], part_dict[j]['eta'], 
                                                                part_dict[j]['phi'],  np.repeat(4e-9, nevents))

        H_candidate = b1 + b2
        invm = H_candidate.mass

        X_temp.append(inputs)
        m_temp.append(invm)
        
m = np.array((m_temp)).T

In [65]:
from consistent_plots import hist

In [66]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib widget

In [67]:
# labels will be used later

labels = ['HX',
 'HX b1, H1 b1',
 'HX b1, H1 b2',
 'HX b1, H2 b1',
 'HX b1, H2 b2',
 'HX b2, H1 b1',
 'HX b2, H1 b2',
 'HX b2, H2 b1',
 'HX b2, H2 b2',
 'H1',
 'H1 b1, H2 b1',
 'H1 b1, H2 b2',
 'H1 b2, H2 b1',
 'H1 b2, H2 b2',
 'H2']

In [68]:
# Let's see what the Higgs masses look like.

bins = np.linspace(50,200,40)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(7,4))

ax = axs[0]
n1, *_ = hist(ax, m[:,0] , bins=bins, label='HX')
n2, *_ = hist(ax, m[:,9] , bins=bins, label='H1')
n3, *_ = hist(ax, m[:,14], bins=bins, label='H2')

ax = axs[1]
n, *_ = ax.hist([m[:,0], m[:,9], m[:,14]], bins=bins, stacked=True, histtype='step', align='mid')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
# Collect Higgs masses 
higgs_mass = np.concatenate((m[:,0], m[:,9], m[:,14]))
hx_mass = m[:,0]
h1_mass = m[:,9]
h2_mass = m[:,14]
bins = np.linspace(90,150,40)
xdata = (bins[1:] + bins[:-1])/2

# Histograms
ydata, _ = np.histogram(higgs_mass, bins=bins)
hxydata, _ = np.histogram(hx_mass, bins=bins)
h1ydata, _ = np.histogram(h1_mass, bins=bins)
h2ydata, _ = np.histogram(h2_mass, bins=bins)

# Normalize for comparison
ydata = ydata / ydata.max()
hxydata = hxydata / hxydata.max()
h1ydata = h1ydata / h1ydata.max()
h2ydata = h2ydata / h2ydata.max()

In [96]:
# Fit the masses with a Gaussian
from lmfit.models import GaussianModel, SkewedGaussianModel

model = GaussianModel()

# create parameters with initial guesses:
params = model.make_params(center=125, amplitude=1, sigma=15)

result = model.fit(ydata, params, x=xdata)
print(result.fit_report())

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 60
    # data points      = 39
    # variables        = 3
    chi-square         = 0.05320934
    reduced chi-square = 0.00147804
    Akaike info crit   = -251.286237
    Bayesian info crit = -246.295552
[[Variables]]
    amplitude:  37.7946583 +/- 0.45995584 (1.22%) (init = 1)
    center:     117.281144 +/- 0.20638438 (0.18%) (init = 125)
    sigma:      15.2313877 +/- 0.22673335 (1.49%) (init = 15)
    fwhm:       35.8671764 +/- 0.53391623 (1.49%) == '2.3548200*sigma'
    height:     0.98992214 +/- 0.01154952 (1.17%) == '0.3989423*amplitude/max(1e-15, sigma)'
[[Correlations]] (unreported correlations are < 0.100)
    C(amplitude, sigma) =  0.645


In [97]:
result.params

name,value,standard error,relative error,initial value,min,max,vary,expression
amplitude,37.7946583,0.45995584,(1.22%),1,-inf,inf,True,
center,117.281144,0.20638438,(0.18%),125,-inf,inf,True,
sigma,15.2313877,0.22673335,(1.49%),15,0.00000000,inf,True,
fwhm,35.8671764,0.53391623,(1.49%),35.3223,-inf,inf,False,2.3548200*sigma
height,0.98992214,0.01154952,(1.17%),0.026596153333333334,-inf,inf,False,"0.3989423*amplitude/max(1e-15, sigma)"


In [98]:
params = {}
for name,param in result.params.items():
    print(name,param.value, param.stderr)
    params[name] = {'value':param.value, 'error':param.stderr}

amplitude 37.79465831097356 0.4599558403971399
center 117.28114441873778 0.20638437874626137
sigma 15.231387706114955 0.22673334884040292
fwhm 35.86717639811362 0.5339162265204757
height 0.9899221400713594 0.01154952425166262


In [104]:
# Here is where the magic happens.

std_dev = 2

# Create a Higgs mass mask
higgs_mask = np.repeat(False, 15)
higgs_mask[0] = True
higgs_mask[9] = True
higgs_mask[14] = True

# Initialize the various higgs masks
contains_0_higgs = np.array((), dtype=bool)
contains_1_higgs = np.array((), dtype=bool)
contains_2_higgs = np.array((), dtype=bool)
contains_3_higgs = np.array((), dtype=bool)

# Define boundaries of mass cut
lower = params['center']['value'] - std_dev*params['sigma']['value']
upper = params['center']['value'] + std_dev*params['sigma']['value']

shift = False
if shift:
    lower += 5
    upper += 5

# Masks for each of the three Higgs'
HX_mask = np.logical_and(m[:,0] > lower, m[:,0] < upper)
H1_mask = np.logical_and(m[:,9] > lower, m[:,9] < upper)
H2_mask = np.logical_and(m[:,14] > lower, m[:,14] < upper)

Y_bool = np.logical_and(H1_mask, H2_mask)
H_bool = np.logical_and(HX_mask, Y_bool)

# This will confirm that the proportion of events in which all three Higgs pass
# is in fact ~70%.
print(np.sum(H_bool)/len(H_bool))

event_indices_passing_pairs = []
n_pairs = np.array(())
Higgs_pairs = np.array(())
nonHiggs_pairs = np.array(())
for ievt in tqdm(range(nevents)):
    mass_cut_mask = np.logical_and(m[ievt,] > lower, m[ievt,] < upper)
    
    arr = mass_cut_mask[higgs_mask]
    contains_0_higgs = np.append(contains_0_higgs, np.sum(arr) == 0)
    contains_1_higgs = np.append(contains_1_higgs, np.sum(arr) == 1)
    contains_2_higgs = np.append(contains_2_higgs, np.sum(arr) == 2)
    contains_3_higgs = np.append(contains_3_higgs, np.sum(arr) == 3)
    
    event_indices_passing_pairs.append(mass_cut_mask)
    n_pairs = np.append(n_pairs, int(np.sum(mass_cut_mask)))
    Higgs_pairs = np.append(Higgs_pairs, int(np.sum(mass_cut_mask[higgs_mask])))
    nonHiggs_pairs = np.append(nonHiggs_pairs, int(np.sum(mass_cut_mask[~higgs_mask])))
    
passing_pairs = np.array((event_indices_passing_pairs))

0.7154841034676063


100%|██████████| 199792/199792 [01:22<00:00, 2436.33it/s]


In [107]:
# Let's see the multiplicity of non-Higgs pairs that pass the mass cut

plt.close()
fig, ax = plt.subplots()
b = np.arange(17)
nH, b = np.histogram(n_pairs, bins=b)
ax.bar(b[:-1], nH)
ax.text(0.55,0.9,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='smaller')
ax.set_xlabel('Number of Pairs Passing Mass Cut')
nH_avg = np.average(n_pairs)
ax.text(0.55,0.84,fr"Average = {nH_avg:.2f} Pairs", transform=ax.transAxes, fontsize='smaller')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.55, 0.84, 'Average = 6.29 Pairs')

In [108]:
# Let's see the multiplicity of non-Higgs pairs that pass the mass cut

plt.close()
fig, ax = plt.subplots()
b = np.arange(14)
nH, b = np.histogram(nonHiggs_pairs, bins=b)
ax.bar(b[:-1], nH)
ax.text(0.55,0.9,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='smaller')
ax.set_xlabel('Number of Non-Higgs Pairs Passing Mass Cut')
nH_avg = np.average(nonHiggs_pairs)
ax.text(0.55,0.84,fr"Average = {nH_avg:.2f} Pairs", transform=ax.transAxes, fontsize='smaller')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.55, 0.84, 'Average = 3.61 Pairs')

In [109]:
N, b = np.histogram(higgs_mass, bins=bins)
y = N / N.max()
err = np.reciprocal(np.sqrt(N))
x = (bins[:-1] + bins[1:])/2

plt.close()
fig, ax = plt.subplots()

for low,mid,high,n,e in zip(bins[:-1],x,bins[1:],y,err):
    ax.plot([low,high], [n,n], color='black')
    ax.plot([mid,mid], [n-e, n+e], color='black')

ax.scatter(xdata, y, s=20, c='black', marker='+', label='Higgs data')
ax.plot(xdata, result.best_fit, label='fit')
ax.set_xlabel(r'$m_H$ [GeV]')
ax.text(0.3, 0.3, fr"$\mu$ = {params['center']['value']:.1f} $\pm$ {params['center']['error']:.1f} GeV", transform=ax.transAxes, color='C0')
ax.text(0.3, 0.23, fr"$\sigma$ = {params['sigma']['value']:.1f} $\pm$ {params['sigma']['error']:.1f} GeV", transform=ax.transAxes, color='C0')
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
from prettytable import PrettyTable

In [78]:
table = PrettyTable()
table.field_names = ['Pair', 'Efficiency']

for i in [0, 9, 14]:
    table.add_row([labels[i], f'{np.around(np.sum(passing_pairs[:,i])/nevents*100, decimals=1)}%'])
    
higgs_masses = np.column_stack((passing_pairs[:,0], passing_pairs[:,9], passing_pairs[:,14]))
table.add_row(['Any Higgs', f'{np.around(np.sum(np.any(higgs_masses, axis=1))/nevents*100, decimals=1)}%'])
table.add_row(['All Higgs', f'{np.around(np.sum(np.all(higgs_masses, axis=1))/nevents*100, decimals=1)}%'])
    
for i,label in enumerate(labels):
    if i == 0 or i == 9 or i == 14: continue
    table.add_row([label, f'{np.around(np.sum(passing_pairs[:,i])/nevents*100, decimals=1)}%'])

In [79]:
print(table)

+--------------+------------+
|     Pair     | Efficiency |
+--------------+------------+
|      HX      |   89.4%    |
|      H1      |   89.7%    |
|      H2      |   88.9%    |
|  Any Higgs   |   99.8%    |
|  All Higgs   |   71.5%    |
| HX b1, H1 b1 |    3.8%    |
| HX b1, H1 b2 |   22.0%    |
| HX b1, H2 b1 |   22.8%    |
| HX b1, H2 b2 |   34.6%    |
| HX b2, H1 b1 |   21.3%    |
| HX b2, H1 b2 |   44.9%    |
| HX b2, H2 b1 |   38.2%    |
| HX b2, H2 b2 |   45.0%    |
| H1 b1, H2 b1 |   11.3%    |
| H1 b1, H2 b2 |   31.8%    |
| H1 b2, H2 b1 |   38.4%    |
| H1 b2, H2 b2 |   46.9%    |
+--------------+------------+


In [110]:
plt.close()
fig, ax = plt.subplots()

# n,   bins = np.histogram(n_pairs, bins=np.arange(0,16))
n_0, bins = np.histogram(nonHiggs_pairs[contains_0_higgs], bins=np.arange(0,13))
n_1, bins = np.histogram(nonHiggs_pairs[contains_1_higgs], bins=np.arange(0,13))
n_2, bins = np.histogram(nonHiggs_pairs[contains_2_higgs], bins=np.arange(0,13))
n_3, bins = np.histogram(nonHiggs_pairs[contains_3_higgs], bins=np.arange(0,13))

width=0.5

ax.bar(bins[:-1], n_0, width, label='0 Higgs')
ax.bar(bins[:-1], n_1, width, bottom=n_0, label='1 Higgs')
ax.bar(bins[:-1], n_2, width, bottom=n_1+n_0, label='2 Higgs')
ax.bar(bins[:-1], n_3, width, bottom=n_1+n_2+n_0, label='3 Higgs')
# ax.bar(bins[:-1]+width/2, n, width, label='No Cuts')

ax.set_title('Multiplicity of Non-Higgs Pairs \nThat Pass Mass Cut Per Event')
ax.set_xlabel('Number of Passing Non-Higgs Pairs')
ax.text(0.62,0.57,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='x-small')
ax.legend(loc=1)

#For each of the bins, plot the real Higgs masses

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
plt.close()
fig, ax = plt.subplots()

# n,   bins = np.histogram(n_pairs, bins=np.arange(0,16))
n_0, bins = np.histogram(nonHiggs_pairs[contains_0_higgs], bins=np.arange(0,17))
n_1, bins = np.histogram(nonHiggs_pairs[contains_1_higgs]+np.ones_like(nonHiggs_pairs[contains_1_higgs]), bins=np.arange(0,17))
n_2, bins = np.histogram(nonHiggs_pairs[contains_2_higgs]+2*np.ones_like(nonHiggs_pairs[contains_2_higgs]), bins=np.arange(0,17))
n_3, bins = np.histogram(nonHiggs_pairs[contains_3_higgs]+3*np.ones_like(nonHiggs_pairs[contains_3_higgs]), bins=np.arange(0,17))

width=0.5

ax.bar(bins[:-1], n_0, width, label='0 Higgs')
ax.bar(bins[:-1], n_1, width, bottom=n_0, label='1 Higgs')
ax.bar(bins[:-1], n_2, width, bottom=n_1+n_0, label='2 Higgs')
ax.bar(bins[:-1], n_3, width, bottom=n_1+n_2+n_0, label='3 Higgs')
# ax.bar(bins[:-1]+width/2, n, width, label='No Cuts')

ax.set_title('Multiplicity of Pairs That \nPass Mass Cut Per Event')
ax.set_xlabel('Number of Passing Pairs')
ax.text(0.6,0.5,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='x-small')
ax.legend(loc=1)

#For each of the bins, plot the real Higgs masses

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
plt.close()
fig, ax = plt.subplots()

# n,   bins = np.histogram(n_pairs, bins=np.arange(0,16))
n_0, bins = np.histogram(nonHiggs_pairs[contains_0_higgs], bins=np.arange(0,13))
n_1, bins = np.histogram(nonHiggs_pairs[contains_1_higgs], bins=np.arange(0,13))
n_2, bins = np.histogram(nonHiggs_pairs[contains_2_higgs], bins=np.arange(0,13))
n_3, bins = np.histogram(nonHiggs_pairs[contains_3_higgs], bins=np.arange(0,13))

width=0.2

ax.bar(bins[:-1]-width*3/2, n_0, width, label='0 Higgs')
ax.bar(bins[:-1]-width/2, n_1, width, label='1 Higgs')
ax.bar(bins[:-1]+width/2, n_2, width, label='2 Higgs')
ax.bar(bins[:-1]+width*3/2, n_3, width, label='3 Higgs')
# ax.bar(bins[:-1]+width/2, n, width, label='No Cuts')

ax.set_title('Multiplicity of Non-Higgs Pairs \nThat Pass Mass Cut Per Event')
ax.set_xlabel('Number of Passing Non-Higgs Pairs')
ax.text(0.55,0.9,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='smaller')
ax.legend(loc=2)

#For each of the bins, plot the real Higgs masses

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
plt.close()
fig, ax = plt.subplots()
ax.set_title('3 passing pairs in the event')
hist(ax, m[:,0][n_pairs==3], bins=np.linspace(75,200,30), label='HX')
hist(ax, m[:,9][n_pairs==3], bins=np.linspace(75,200,30), label='H1')
hist(ax, m[:,14][n_pairs==3], bins=np.linspace(75,200,30), label='H2')
ax.plot([90,90],[0,500], c='black')
ax.plot([150,150],[0,500], c='black')
ax.text(0.55,0.95,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='smaller')
ax.set_xlabel(r'$m_H$ [GeV]')
ax.legend(loc=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
len(X_temp)

15

In [85]:
plt.close()
fig, ax = plt.subplots()
hist(ax, X_temp[0][:,0][n_pairs==3], label='HX b1 pt', bins=np.linspace(0,400,60))
hist(ax, X_temp[9][:,0][n_pairs==3], label='H1 b1 pt', bins=np.linspace(0,400,60))
hist(ax, X_temp[14][:,0][n_pairs==3], label='H2 b1 pt', bins=np.linspace(0,400,60))
ax.text(0.55,0.95,fr"{lower:.1f} GeV < $m_\mathrm{{inv}}$ < {upper:.1f} GeV", transform=ax.transAxes, fontsize='smaller')
ax.set_xlabel(r'$p_T$ [GeV]')
ax.legend(loc=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
table2 = PrettyTable()

n_tot = n_0+n_1+n_2+n_3

table2.field_names = ['Num Pairs', 'Num Total Events in Bin', '% 0 H', '% 1 H', '% 2 H', '% 3 H']

for i,n in enumerate(n_tot):
    table2.add_row([i, n, round(n_0[i]/n*100, 1), round(n_1[i]/n*100, 1), round(n_2[i]/n*100, 1), round(n_3[i]/n*100, 1)])
    
print(table2)

+-----------+-------------------------+-------+-------+-------+-------+
| Num Pairs | Num Total Events in Bin | % 0 H | % 1 H | % 2 H | % 3 H |
+-----------+-------------------------+-------+-------+-------+-------+
|     0     |           1804          |  0.4  |  4.2  |  36.4 |  59.0 |
|     1     |          11478          |  0.2  |  4.2  |  28.3 |  67.3 |
|     2     |          32594          |  0.1  |  3.2  |  24.2 |  72.4 |
|     3     |          50779          |  0.1  |  2.8  |  22.8 |  74.3 |
|     4     |          49799          |  0.1  |  2.9  |  23.5 |  73.5 |
|     5     |          33275          |  0.2  |  3.5  |  25.6 |  70.8 |
|     6     |          14802          |  0.2  |  3.9  |  29.4 |  66.5 |
|     7     |           4317          |  0.3  |  5.8  |  36.5 |  57.3 |
|     8     |           820           |  0.5  |  9.4  |  47.2 |  42.9 |
|     9     |           119           |  1.7  |  22.7 |  62.2 |  13.4 |
|     10    |            5            |  0.0  |  60.0 |  40.0 | 

<ipython-input-86-8013785530ee>:8: RuntimeWarning: invalid value encountered in long_scalars
  table2.add_row([i, n, round(n_0[i]/n*100, 1), round(n_1[i]/n*100, 1), round(n_2[i]/n*100, 1), round(n_3[i]/n*100, 1)])


In [87]:
plt.close()
fig, ax = plt.subplots()

n_1, bins, _ = hist(ax, m[:,0][H_bool])
n_2, bins, _ = hist(ax, m[:,9][H_bool])
n_3, bins, _ = hist(ax, m[:,14][H_bool])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 

In [88]:
plt.close()
fig, ax = plt.subplots()

s = ax.scatter(m[:,0][n_pairs==3], m[:,9][n_pairs==3], s=1, c=m[:,14][n_pairs==3], cmap='rainbow')
ax.plot([0, 400], [90, 90])
ax.plot([0, 400], [150, 150])

ax.plot([90, 90]  , [0, 400])
ax.plot([150, 150], [0, 400])

ax.set_ylim(0,400)
ax.set_xlim(0,400)
plt.colorbar(s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
plt.close()
fig, ax = plt.subplots()

bins=np.arange(16)

n_1, bins, _ = hist(ax, n_pairs[H_bool], bins=bins)
n_2, bins, _ = hist(ax, n_pairs, bins=bins)

ax = ax.twinx()
x = (bins[:-1] + bins[1:])/2
y = np.where(n_2!=0, n_1/n_2, 0)
ax.scatter(x, y, color='C2')
ax.set_ylim(0,1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-89-437523f825a6>:11: RuntimeWarning: invalid value encountered in true_divide
  y = np.where(n_2!=0, n_1/n_2, 0)


(0.0, 1.0)

In [90]:
plt.close()
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

ax = axs[0]
s = ax.scatter(m[:,0][n_pairs==3], m[:,9][n_pairs==3], s=1)
bool1 = np.logical_and(m[:,0][n_pairs==3] > lower, m[:,0][n_pairs==3] < upper)
bool2 = np.logical_and(m[:,9][n_pairs==3] > lower, m[:,9][n_pairs==3] < upper)
bool_total1 = bool1 & bool2
print(np.sum(bool_total1)/len(m[:,0][n_pairs==3]))
ax.plot([0, 400], [90, 90])
ax.plot([0, 400], [150, 150])

ax.plot([90, 90]  , [0, 400])
ax.plot([150, 150], [0, 400])

ax.set_ylim(0,400)
ax.set_xlim(0,400)

ax = axs[1]
s = ax.scatter(m[:,9][n_pairs==3], m[:,14][n_pairs==3], s=1)
bool1 = np.logical_and(m[:,14][n_pairs==3] > lower, m[:,14][n_pairs==3] < upper)
bool2 = np.logical_and(m[:,9][n_pairs==3] > lower, m[:,9][n_pairs==3] < upper)
bool_total2 = bool1 & bool2
print(np.sum(bool_total2)/len(m[:,0][n_pairs==3]))
ax.plot([0, 400], [90, 90])
ax.plot([0, 400], [150, 150])

ax.plot([90, 90]  , [0, 400])
ax.plot([150, 150], [0, 400])

ax.set_ylim(0,400)
ax.set_xlim(0,400)

ax = axs[2]
s = ax.scatter(m[:,14][n_pairs==3], m[:,0][n_pairs==3], s=1)
bool1 = np.logical_and(m[:,0][n_pairs==3] > lower, m[:,0][n_pairs==3] < upper)
bool2 = np.logical_and(m[:,14][n_pairs==3] > lower, m[:,14][n_pairs==3] < upper)
bool_total3 = bool1 & bool2
print(np.sum(bool_total3)/len(m[:,0][n_pairs==3]))
ax.plot([0, 400], [90, 90])
ax.plot([0, 400], [150, 150])

ax.plot([90, 90]  , [0, 400])
ax.plot([150, 150], [0, 400])

ax.set_ylim(0,400)
ax.set_xlim(0,400)

print(bool_total1*bool_total2*bool_total3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.4616519174041298
0.3746312684365782
0.35176991150442477
[False False False ... False False  True]
